In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

![inception module](inception_1.png "Inception module")

In [2]:
class InceptionBlock(nn.Module):
    
    def __init__(self, in_0, out_1, out_2_1, out_2_2, out_3_1, out_3_2, out_4_2):
        super().__init__()
        self.conv_1 = nn.Conv2d(in_0, out_1, kernel_size=1)
        self.conv_2_1 = nn.Conv2d(in_0, out_2_1, kernel_size=1)
        self.conv_2_2 = nn.Conv2d(out_2_1, out_2_2, kernel_size=3, padding=1)
        self.conv_3_1 = nn.Conv2d(in_0, out_3_1, kernel_size=1)
        self.conv_3_2 = nn.Conv2d(out_3_1, out_3_2, kernel_size=5, padding=2)
        self.pool_4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_4_2 = nn.Conv2d(in_0, out_4_2, kernel_size=1)
        
    def forward(self, x):
        path_1 = self.conv_1(x)
        path_1 = F.relu(path_1)
        
        path_2 = self.conv_2_1(x)
        path_2 = F.relu(path_2)
        path_2 = self.conv_2_2(path_2)
        path_2 = F.relu(path_2)
        
        path_3 = self.conv_3_1(x)
        path_3 = F.relu(path_3)
        path_3 = self.conv_3_2(path_3)
        path_3 = F.relu(path_3)
        
        path_4 = self.pool_4_1(x)
        path_4 = self.conv_4_2(path_4)
        path_4 = F.relu(path_4)
        
        x = torch.cat((path_1, path_2, path_3, path_4), dim=1)
        
        return x

In [3]:
sample_inception_block_input = torch.randn(1, 192, 28, 28)
sample_inception_block_input.shape

torch.Size([1, 192, 28, 28])

In [4]:
#inception block works fine
test_inception_block = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
test_inception_block.forward(sample_inception_block_input)

tensor([[[[0.0000, 0.0370, 0.7375,  ..., 0.0000, 0.5224, 0.4864],
          [0.0000, 0.2402, 0.0796,  ..., 0.0000, 0.0000, 0.4021],
          [0.1581, 0.0000, 0.0000,  ..., 0.0000, 0.2499, 1.1586],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.8399, 0.7623],
          [0.5253, 0.0000, 0.0000,  ..., 0.0000, 0.4505, 0.4039],
          [0.8262, 0.0830, 0.0000,  ..., 0.0000, 0.0000, 0.2985]],

         [[0.1300, 0.0000, 0.0000,  ..., 0.0000, 0.2802, 0.0000],
          [0.0000, 0.0638, 0.0000,  ..., 0.1772, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.7269,  ..., 0.8356, 0.0000, 0.4428],
          ...,
          [0.4667, 0.9447, 0.2252,  ..., 0.0000, 0.0000, 0.0000],
          [0.2258, 0.0923, 0.0000,  ..., 0.6985, 0.8805, 0.0000],
          [0.0609, 0.4870, 0.0000,  ..., 0.0000, 0.6240, 0.0000]],

         [[0.3156, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2950],
          [1.0187, 0.1072, 0.0000,  ..., 0.2558, 0.4695, 0.0000],
          [0.0000, 0.1695, 0.0588,  ..., 0

![inception architecture](inception_2.png "Inception architecture")

In [5]:
class InceptionNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(3, 64, 7, stride=2, padding=3)
        self.conv_2 = nn.Conv2d(64, 192, 3, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inc_block_3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inc_block_3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.inc_block_4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inc_block_4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inc_block_4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inc_block_4d = InceptionBlock(512, 256, 160, 320, 32, 128, 128)
        self.inc_block_4e = InceptionBlock(512, 256, 160, 320, 32, 128, 128)
        self.inc_block_5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inc_block_5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout2d(0.4)
        self.fc_1 = nn.Linear(1024, 128)
        self.fc_2 = nn.Linear(128, 16)
        self.fc_3 = nn.Linear(16, 1)
           
        
    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = self.max_pool(x)
        x = F.relu(self.conv_2(x))
        x = self.max_pool(x)
        x = self.inc_block_3a(x)
        x = self.inc_block_3b(x)
        x = self.max_pool(x)
        x = self.inc_block_4a(x)
        x = self.inc_block_4b(x)
        x = self.inc_block_4c(x)
        x = self.inc_block_4d(x)
        x = self.max_pool(x)
        x = self.inc_block_5a(x)
        x = self.inc_block_5b(x)
        x = self.avg_pool(x)
        x = self.dropout(x).view(-1, 1024)
        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x))
        x = F.relu(self.fc_3(x))   

        return x

In [6]:
#test
model = InceptionNet()
sample_inception_model_input = torch.randn(32, 3, 224, 224)
model.forward(sample_inception_model_input).size()

torch.Size([32, 1])